In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18

In [3]:
import torch
from simpletransformers.t5 import T5Model, T5Args

In [4]:
cuda =  torch.cuda.is_available()
cuda

True

In [5]:
import pandas as pd

train = pd.read_csv('train.csv')
valid  = pd.read_csv('val.csv')

In [6]:
train = train.rename(columns={"summary":"target_text", "text":"input_text"})
valid = valid.rename(columns={"summary":"target_text", "text":"input_text"})

train['prefix'] = ''
valid['prefix'] = ''

In [7]:
train

,input_text,target_text,prefix
0,"from lalit k jha washington, apr (pti) desc...","imf chief christine lagarde has said, ""the gst...",
1,they look like premium rado watches at first g...,"several copies of rado watches, sunglasses, an...",
2,"in his bitter and long tussle with the centre,...","the supreme court, while hearing an aap govern...",
3,shahid kapoor's little sister sanah kapoor is ...,"as per reports, shahid kapoor's sister sanah k...",
4,in what only brings to the fore sheer brazenne...,"telugu desam party mla jaleel khan, who repres...",
...,...,...,...
3159,no one knew who jammu and kashmir police offic...,the jammu and kashmir police on monday said it...,
3160,"noida-based startup ringing bells, infamous fo...","noida-based company, which claims its phone fr...",
3161,test captain virat kohli has emerged as india'...,virat kohli was named india's sportsperson of ...,
3162,the supreme court today refused to give an urg...,the supreme court has refused to give an urgen...,


In [8]:
valid

,input_text,target_text,prefix
0,"new delhi, jul (pti) no proposal is under c...",the centre informed the lok sabha there is no ...,
1,"mumbai, jul (pti) farmers from a village in...",farmers from a village in maharashtra have lod...,
2,"director: ashtar sayed cast: raveena tandon, m...",raveena tandon's 'maatr' which released on fri...,
3,"aiadmk amma leader ttv dinakaran, who is accus...",nephew of jailed aiadmk general secretary vk s...,
4,"new delhi, jun (pti) attorney general mukul...",indian attorney general mukul rohatgi has aske...,
...,...,...,...
347,the raghubar das government in jharkhand on tu...,the jharkhand government on tuesday cleared a ...,
348,nearly a week after eight amarnath yatris were...,jammu police have said that a tyre burst led t...,
349,"it's the season of clashes, and the next time ...",actor shah rukh khan has said that he wanted t...,
350,russia's mariya savinova-farnosova has been st...,russia's mariya savinova-farnosova has been st...,


In [9]:
train_params = {
    'max_seq_length': 512,
    'max_length': 128,
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'num_train_epochs': 2,
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 10000,
    'use_multiprocessing': False,
    'fp16': False,
    'save_steps': -1,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    'no_cache': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'preprocess_inputs': False,
    'num_return_sequences': 1
}

In [10]:
model = T5Model('t5', 't5-small', args=train_params, use_cuda=cuda)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
import gc

In [12]:
model.train_model(train, eval_data=valid)
gc.collect()

  0%|          | 0/3164 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/396 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Running Epoch 2 of 2:   0%|          | 0/396 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/352 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


3687